In [7]:
from utils.utils import make_patches, update_coordinates
from ultralytics import YOLO
from PIL import Image
import os
import pandas as pd

# Patches parameters
data_tif_path = "Datasets/Pipeline/Crow_traps_Original"
data_patches_path = "Datasets/Pipeline/Crow_traps_Original/Patches"
patch_size = 224

# Classification parameters
model_cls_path = "runs/classify/train/weights/best.pt"
data_out_cls_path = "Datasets/Pipeline/Classified"

if not os.path.exists(data_out_cls_path):
    os.makedirs(data_out_cls_path)

# Detection parameters
model_det_path = "runs/detect/train_30_general/weights/best.pt"
data_out_det_path = "Datasets/Pipeline"

# Coordinate parameters
coordinates = {}
csv_out_path = "Datasets/Pipeline/coordinates.csv"

In [2]:
# Make patches

patches = make_patches(data_tif_path, data_patches_path, patch_size=patch_size)
print("Total patches: ", len(patches))

N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (2, 1, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
N patches  (4, 3, 1, 224, 224, 3)
Total patches:  122


In [8]:
# Classify patches

model_cls = YOLO(model_cls_path)  # load model
results_cls = model_cls.predict(source=data_patches_path, imgsz=patch_size, conf=0.5) 

# Save results
for r in results_cls:
    if r.probs.top1==1:
        # Save imgs for Detector
        im = Image.fromarray(r.orig_img[..., ::-1])  # RGB PIL image
        im.save(os.path.join(data_out_cls_path,os.path.basename(r.path)))

        # Update coordinates dictionary
        coordinates = update_coordinates(coordinates, r.path, data_tif_path, "area", patch_size=patch_size)

# Detect crow_traps

model_dtc = YOLO(model_det_path)  # load model
results_dtc = model_dtc.predict(source=data_out_cls_path, save=True, project=data_out_det_path, name="Detected",imgsz=patch_size, conf=0.5, classes=0) 

for r in results_dtc:

    boxes = r.boxes.xywh.cpu().numpy()

    for box in boxes:
    
        # Update coordinates dictionary
        coordinates = update_coordinates(coordinates, r.path, data_tif_path, "crow_trap", (box[0], box[1]), patch_size=patch_size)



image 1/122 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Crow_traps_Original/Patches/100631_0_0.jpg: 224x224 background 0.99, crow_trap 0.01, 2.5ms
image 2/122 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Crow_traps_Original/Patches/100631_0_1.jpg: 224x224 background 0.79, crow_trap 0.21, 4.0ms
image 3/122 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Crow_traps_Original/Patches/100631_0_2.jpg: 224x224 background 1.00, crow_trap 0.00, 2.5ms
image 4/122 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Crow_traps_Original/Patches/100631_1_0.jpg: 224x224 background 1.00, crow_trap 0.00, 2.6ms
image 5/122 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Crow_traps_Original/Patches/100631_1_1.jpg: 224x224 background 1.00, crow_trap 0.00, 2.3ms
image 6/122 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Crow_traps_Original/Patches/


101343_2_1 @ -4.051563270693071,57.09718259638177

103138_2_1 @ -3.994065088495014,55.16442061499526

103519_2_1 @ -1.716027820063532,54.18657383184679

103849_0_1 @ -2.431714556218832,55.750860490104444

103849_2_1 @ -2.431705656863632,55.75005510165397

104268_2_1 @ -3.9916301669756704,55.41827344718695

104270_2_1 @ -3.9744447894927375,55.4237656447266



image 1/7 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Classified/101343_2_1.jpg: 224x224 1 crow_trap, 7.0ms
image 2/7 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Classified/103138_2_1.jpg: 224x224 1 crow_trap, 6.7ms
image 3/7 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Classified/103519_2_1.jpg: 224x224 1 crow_trap, 6.4ms
image 4/7 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Classified/103849_0_1.jpg: 224x224 (no detections), 5.7ms
image 5/7 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Classified/103849_2_1.jpg: 224x224 1 crow_trap, 6.3ms
image 6/7 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Classified/104268_2_1.jpg: 224x224 (no detections), 5.7ms
image 7/7 /home/davide/Desktop/Projects/RSPB-Crow-traps-detection/Datasets/Pipeline/Classified/104270_2_1.jpg: 224x224 (no detections), 5.9ms
Speed: 0.5ms preproce


101343_2_1 @ -4.051272960625111,57.09728986411098

103138_2_1 @ -3.9938017969338486,55.16452181996046

103519_2_1 @ -1.7157652539070192,54.186661697027965

103849_2_1 @ -2.4318379778498804,55.75005165502541


In [9]:
df = pd.DataFrame.from_dict(coordinates, orient="index", columns=["Coordinates_QGIS", "Type", "Status"])
df.to_csv(csv_out_path)
df

,Coordinates_QGIS,Type,Status
101343_2_1,"-4.051272960625111,57.09728986411098",crow_trap,to_check
103138_2_1,"-3.9938017969338486,55.16452181996046",crow_trap,to_check
103519_2_1,"-1.7157652539070192,54.186661697027965",crow_trap,to_check
103849_0_1,"-2.431714556218832,55.750860490104444",area,to_check
103849_2_1,"-2.4318379778498804,55.75005165502541",crow_trap,to_check
104268_2_1,"-3.9916301669756704,55.41827344718695",area,to_check
104270_2_1,"-3.9744447894927375,55.4237656447266",area,to_check
